In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-16:] == 'image-to-tex-OCR':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR/Jupyter_Notebooks
Current path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR


In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
from Data.Data_Module_CFG import Data_Module_CFG
from Models.Printed_Tex_Transformer import ResNetTransformer
from Lightning_Models.Printed_Tex_Lit_Model import LitResNetTransformer
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from PIL import Image
import torch
from pytorch_lightning.loggers import TensorBoardLogger
from torchvision import transforms
from PIL import Image
from IPython.display import display, Math
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps")
torch.cuda.is_available()

False

In [4]:


class CFG:
    
    device = dev
    
   
    use_wandb = True
    
    ######### Dataset #########
    
    # stage: 'train' or 'test'
    stage = 'train'
    set_max_label_length = 96
    number_png_images_to_use_in_dataset=180*1000
    labels_transform='default'
    image_transform_name ='alb'
    train_val_fraction = 0.95
    vocabulary_path = 'Tokenizer/230k_ver2.json'
    load_vocabulary = True
    image_padding = True
    max_width = 100*1000


    batch_size = 32
    num_workers = 0
    on_gpu = False
    
    ######## Model and Lightning Model paramters ############
    
    # MODEL
    
    
    
    
    
    checkpoint = None
    save_directory = None
    
    
    accumulate_grad_batches = 64 // batch_size  # experiments showed batch_size * accumulate_grad = 192 is optimal
    learning_rate =  1e-5
    eta_min = 1e-8
    t_max = 40
    max_epochs = 35
    weight_decay =  1e-5
    precision =16
    
    # checkpointing
    save_top_k=5
    
    monitor="FBETA"
    mode="max"
    
    
    ####### Augemtnations ###############
    
    # Training Aug
    train_transforms = [
       A.augmentations.geometric.resize.Resize(height=64, width= 512, p=1),
                        #A.ShiftScaleRotate(shift_limit=0, scale_limit=(-.15, 0), rotate_limit=1, border_mode=0, interpolation=3, value=[255, 255, 255], p=1),
                        #A.Affine(scale=(0.6, 1.0), rotate=(-2, 2), cval=255, p=0.5),
                        #A.GridDistortion(distort_limit=0.1, border_mode=0, interpolation=3, value=[255, 255, 255], p=.5),
                        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
                        A.GaussianBlur(blur_limit=(1, 1), p=0.5),
                        A.RandomBrightnessContrast(.5, (-.5, .5), True, p=0.3),
                        #A.ImageCompression(95, p=.3),
                        ToTensorV2(),
                        ]
    

    
    # Validaiton Aug
    val_transforms = [
        #A.Resize(patch_size, patch_size),
    A.augmentations.geometric.resize.Resize(height=64, width= 512, p=1),

       
        ToTensorV2(),
    ]
    
    # Test Aug
    test_transforms = [
        #A.Resize(patch_size, patch_size),
      

        ToTensorV2(),
    ]
        
    
    

In [5]:
# Generate Data Module by uploading images and formulas
# images need to be in the folder Data/Data_Bank/generated_png_images
# formulas need to be in Data/Data_Bank/final_png_formulas.txt
# image filenames need to be in Data/Data_Bank/corresponding_png_images.txt

dataset = Data_Module_CFG(cfg=CFG
                )


# Generates DataFrame and vocabulary, tokezniers etc
#dataset.prepare_data()

# prepeares the dataloaders with transformations and splits train/val
#dataset.setup(stage = 'train')

171000 9000
                                                  formula  \
50      S = M _ { P } ^ { D - 2 } \int d ^ { D } x \sq...   
86      ( f ) _ { 0 2 } = \frac { \beta \delta ^ { 3 }...   
93      \tilde { m } _ { 0 } = m _ { 0 } \left[ 1 - \l...   
100     \left( f ^ { \prime } , g ^ { \prime } \right)...   
116     { \cal L } = - V ( \varphi ) K \left( { \frac ...   
...                                                   ...   
213411  { \cal H } _ { \ell , 0 } \, : \qquad | \ell ,...   
213413  B ( \psi ) = \left\{ \chi ( s ) \mid [ \chi ( ...   
213414  \delta _ { \alpha } C _ { i } = D _ { i } [ C ...   
213415  s ( A , B , X ) = ( 1 - ( - 1 ) ^ { l e n ( A ...   
213416  [ L _ { m } ^ { M } , G _ { r } ] = \left( \fr...   

                 image_name  height  width  formula_length  aspect_ratio  \
50      30008ba8ff31950.png     128   1113             142             8   
86      8cd74557bfcde8a.png     128    374             104             2   
93      0e3f7126df53660.png

In [6]:
print( 
    'Vocabulary size:',len(dataset.vocabulary),
    "\nTrain-Dataset size:", len(dataset.data_train),
    "\nValidation-Dataset size:", len(dataset.data_val),
    '\nMax label length:', dataset.max_label_length,
    "\nStart <S> goes to index ",dataset.vocabulary['<S>'],
      "\nEnd <E> goes to index ",dataset.vocabulary['<E>'],
      "\nPadding <P> goes to index ",dataset.vocabulary['<P>'],
)

Vocabulary size: 581 
Train-Dataset size: 172342 
Validation-Dataset size: 35882 
Max label length: 98 
Start <S> goes to index  0 
End <E> goes to index  1 
Padding <P> goes to index  2


In [7]:
dataset.df.head()

,formula,image_name,height,width,formula_length,aspect_ratio,latex_tokenized,tokenized_len
0,\gamma _ { C } ( g _ { \mathrm { R } } ) = - \...,8c2ef650d0104e0.png,128,515,151,4,"[\gamma, _, {, C, }, (, g, _, {, \mathrm, {, R...",58
1,G ( q ^ { 2 } ) = i \frac { \alpha h - g } { \...,283d9468d1d7ebc.png,128,483,111,3,"[G, (, q, ^, {, 2, }, ), =, i, \frac, {, \alph...",40
2,w ^ { a b } = w ^ { a b } ( e ) + \kappa ^ { a...,f333d8cf4a4c201.png,128,909,56,7,"[w, ^, {, a, b, }, =, w, ^, {, a, b, }, (, e, ...",26
3,E = \frac { 1 } { 4 G _ { 4 } } \sum _ { i } Q...,a03f7b7a61e5c89.png,128,687,134,5,"[E, =, \frac, {, 1, }, {, 4, G, _, {, 4, }, },...",58
4,\left( v _ { \xi } \right) _ { \ \alpha } ^ { ...,9631cf277eca635.png,128,536,98,4,"[\left(, v, _, {, \xi, }, \right), _, {, \, \a...",29
...,...,...,...,...,...,...,...,...
213411,"{ \cal H } _ { \ell , 0 } \, : \qquad | \ell ,...",8d620a7205499d7.png,128,1912,157,14,"[{, \cal, H, }, _, {, \ell, ,, 0, }, \,, :, \q...",55
213413,B ( \psi ) = \left\{ \chi ( s ) \mid [ \chi ( ...,adeaaf30ac8c3b8.png,128,1824,110,14,"[B, (, \psi, ), =, \left\{, \chi, (, s, ), \mi...",40
213414,\delta _ { \alpha } C _ { i } = D _ { i } [ C ...,c840d2aa9c10c4e.png,128,715,56,5,"[\delta, _, {, \alpha, }, C, _, {, i, }, =, D,...",21
213415,"s ( A , B , X ) = ( 1 - ( - 1 ) ^ { l e n ( A ...",36ef41060a9aa7c.png,128,2931,144,22,"[s, (, A, ,, B, ,, X, ), =, (, 1, -, (, -, 1, ...",72


In [8]:
new 

NameError: name 'new' is not defined

In [ ]:
# Helper Function to convert prediction labels to strings
def token_to_strings(tokens):
    mapping = dataset.vocabulary
    inverse_mapping =dataset.inverse_vocabulary
    s=''
    if tokens.shape[0] ==1:
        tokens = tokens[0]
    for number in tokens:
        letter = inverse_mapping[number.item()]
        s= s + str(letter)
    return s





In [ ]:

transform = transforms.ToPILImage()
display(transform(dataset.data_train[1][0]))
print(token_to_strings(dataset.data_train[1][1]))

In [ ]:
transform = transforms.ToPILImage()
display(transform(dataset.data_val[10][0]))
print(token_to_strings(dataset.data_val[10][1]))

In [ ]:
dataset.data_val.dataframe

In [ ]:
# Initiate the model
model = ResNetTransformer(dataset=dataset).to(dev)

In [ ]:
torch.set_num_threads(1)
logger = TensorBoardLogger("Models_Parameters_Log", name="Test_Run")
lit_model = LitResNetTransformer(model=model)

#use the lit_model to run with Trainer
trainer = Trainer(accelerator="mps",   max_epochs=12, logger=logger,) #accumulate_grad_batches=2, overfit_batches=1, default_root_dir="lightning_logs/ResNet/checkpoints"  
trainer.fit(model=lit_model, datamodule=dataset,)

#add parameter below to trainer.fit to continue from a checkpoint
#ckpt_path='lightning_logs/Character_tokenizer_1/version_2/checkpoints/epoch=0-step=1294.ckpt'
 #ckpt_path='lightning_logs/Character_tokenizer_1/version_4/checkpoints/epoch=15-step=20704.ckpt' )

In [ ]:
# save model
torch.save(model.state_dict(), 'Models_Parameters_Log/first_test.pth')


In [ ]:
import json

def save_dic(dic):
    filename = 'lightning_logs/256_character_1.json'
    with open(filename, 'w') as f:
        f.write(json.dumps(dic, default=str))

def load_dic(filename):
    with open(filename) as f:
        dic = json.loads(f.read())
    return dic

In [ ]:
# Helper Function to convert prediction labels to strings
def token_to_strings(tokens):
    mapping = dataset.vocabulary
    inverse_mapping =dataset.inverse_vocabulary
    s=''
    if tokens.shape[0] ==1:
        tokens = tokens[0]
    for number in tokens:
        letter = inverse_mapping[number.item()]
        s= s +" " + str(letter)
    return s


In [ ]:
# Get image and label from train data -- change number for different ones
image_tensor, label = dataset.data_train[12]
transform = transforms.ToPILImage()
image = transform(image_tensor)

print('\nOriginal image and formula:')
display(image)
print(token_to_strings(label))
print('\nPredicted formula: \n')
with torch.no_grad():
    model.to('cpu')
    model.eval()
    prediction =  model.predict(image_tensor.unsqueeze(0).to('cpu'))
print(token_to_strings(prediction),'\n')
print(display(Math(token_to_strings(prediction))))

In [ ]:
data